# Lab 3.1 - Algorytmy genetyczne
## Dawid Przybyliński 298836

In [1]:
import numpy as np
import random

In [2]:
class evolution:
    
    def __init__(self, population_size, chr_size, target_convert_function, init_method = 'binary'):
        self.n = population_size
        self.m = chr_size
        self.target = target_convert_function
        self.best = None
        
        # initialize random generation
        if init_method == 'binary': # random bits
            self.generation = np.random.choice([0, 1], size=(self.n, self.m), p=[0.5,0.5])
        else: # random uniforms 
            self.generation = np.random.uniform(-8, 8, (self.n, self.m))
            
    
    def evolve(self):
        """
        główna funkcja przeprowadzająca ewolucję i szukająca minimum
        """
        iteration_counter = 0
        current_best_value = -1
        while True:
            
            # check stop condition 
            if iteration_counter == 100:
                return self.target(self.best, arguments = True)
            iteration_counter += 1
            
            # crossover
            parents = self.generation[np.random.choice(self.n, int(np.floor(0.7*self.n)), replace=False), :]
            crossed = self.crossover(parents)
            population = np.vstack((self.generation, crossed))
            population_size = population.shape[0]
            
            # mutation
            mutated = population[np.random.choice(population_size, int(np.floor(0.2*population_size)), replace=False), :]
            mutated = self.mutate(mutated)
            population = np.vstack((population, mutated))
            
            # evaluation
            values = np.asarray([self.target(osobnik) for osobnik in population])
            best_value_index = np.argmax(values)
            if values[best_value_index] > current_best_value:
                self.best = population[best_value_index]
                current_best_value = values[best_value_index]
            
            # selection
            prob = values/values.sum()
            choices = np.random.choice([i for i in range(population.shape[0])], size=self.n, p=prob)
            self.generation = np.asarray([population[choice] for choice in choices])
            
            
            
    def crossover(self,parents):
        """krzyżowanie"""
        children = []
        for i in range(0, parents.shape[0]-1, 2):
            [child_1, child_2] = self.cross(parents[i], parents[i+1])
            children.append(child_1)
            children.append(child_2)
        return np.asarray(children)
            
            
    def cross(self,x,y):
        """
        krzyżowanie jednopunktowe, punkt podziału jest losowy z przedziału [1,len-1]
        """
        split = random.randint(1,len(x)-1)
        child_1 = np.concatenate((x[0:split], y[split:]))
        child_2 = np.concatenate((y[0:split], x[split:]))
        return [child_1, child_2]
    
    def mutate(self, to_mutate):
        """
        mutacja gaussowska
        """
        for i, osobnik in enumerate(to_mutate):
            for j, gen in enumerate(osobnik):
                to_mutate[i][j] += np.random.normal(0,1)
        return to_mutate


In [3]:
def target_1(x,y,z):
    return (x**2) + (y**2) + 2*(z**2)

In [4]:
def eval_1(osobnik, arguments = False):
    target = target_1(osobnik[0],osobnik[1],osobnik[2])
    if arguments:
        return (osobnik[0],osobnik[1],osobnik[2],1/target)
    return 1/target   

In [5]:
for i in range(10):
    evo = evolution(100, 3, eval_1, init_method='dupa')
    print(evo.evolve())

(0.007245258697386271, 0.048209666541406626, -0.053714040412544684, 122.74363450637637)
(-0.030719052990118872, 0.05676751102576917, -0.05201638051558449, 104.41009217790722)
(0.019744972770677993, 0.03351840269971085, -0.035046409549329915, 251.89875526756558)
(-0.014841339812199678, 0.02534550203713315, 0.0761646794282651, 80.22606662149683)
(0.018460876140750926, -0.00453588409163877, -0.03286574091072425, 396.5591288351484)
(-0.015722347155317618, 0.005906966441512118, -0.016901421856784893, 1171.7826660313074)
(0.001818851399146837, -0.0150200884374221, 0.009439929173144748, 2456.1829086181197)
(-0.08284147016192611, 0.014478905572913536, -0.0028658448639508408, 141.06811662431062)
(0.02413029281544139, -0.10327177974710458, -0.04883491494138808, 62.43354979496416)
(0.030825791737830843, -0.0640719122321091, 0.0366674726734325, 129.12478675644587)


In [6]:
def rastrigin_5(x, A=10):
    """funckja rastrigina"""
    return (A*len(x) + np.sum(x**2 - A*np.cos(2*np.pi*x)))

In [7]:
rastrigin_5(np.array([-10,-10,-10,-10,-10]))

500.0

In [8]:
def eval_2(osobnik, arguments = False):
    target = rastrigin_5(osobnik)
    if arguments:
        return (osobnik, 600-target)
    return 600 - target   

In [9]:
evo = evolution(300, 5, eval_2, init_method='dupa')
evo.evolve()

(array([ 0.04891042, -1.03073213, -0.86376829,  0.07385537,  1.04936999]),
 591.4481952659746)